In [28]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import numpy as np

In [35]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

#headers, bot oldugumuzu anlamasın diye

In [55]:
TOTAL_LINK = []
for totalLink in range(1,16):
    TOTAL_LINK.append("https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa="+str(totalLink))
print(TOTAL_LINK)

['https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=1', 'https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=2', 'https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=3', 'https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=4', 'https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=5', 'https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=6', 'https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=7', 'https://www.hepsiburada.com/ara?q=no-frost%20buzdolab%C4%B1&kategori=2147483637_235604&filtreler=fiyat:2500-max&sayfa=8', 'https://www.he

In [66]:
ALL_PRODUCT_URL = []
for products in TOTAL_LINK[::]:
    html= getAndParseURL(products)
    for link in html.findAll("li",{"class":"productListContent-item"}):
        ALL_PRODUCT_URL.append("https://www.hepsiburada.com/"+link.a["href"])
ALL_PRODUCT_URL


['https://www.hepsiburada.com//samsung-rt46k6000ww-tr-no-frost-buzdolabi-p-MTSAMRT46K6000WWTR?magaza=Hepsiburada',
 'https://www.hepsiburada.com//vestel-no-frost-kombi-buzdolabi-nfk48001-x-p-HBCV0000043SAD?magaza=ALISVERİSAVM',
 'https://www.hepsiburada.com//vestel-nf45001-no-frost-buzdolabi-p-HBCV000004EJI3?magaza=Zirve - Vestel Yetkili Online Bayi',
 'https://www.hepsiburada.com//profilo-bd2155wfnn-453-l-no-frost-cift-kapili-buzdolabi-p-HBCV00000OTS59?magaza=BİRTEK AVM',
 'https://www.hepsiburada.com//bosch-kgn86aif0n-682-lt-no-frost-buzdolabi-p-HBV00000VF5J6?magaza=indirim59',
 'https://www.hepsiburada.com//samsung-rt46k6000s8-tr-no-frost-buzdolabi-p-MTSAMRT46K6000S8TR?magaza=Hepsiburada',
 'https://www.hepsiburada.com//profilo-bd3086wfdn-682-lt-no-frost-buzdolabi-p-HBV00000VBM17?magaza=siparişkapıda',
 'https://www.hepsiburada.com//regal-nf-45010-402-lt-no-frost-buzdolabi-p-HBCV000004J8SU?magaza=Hepsiburada',
 'https://www.hepsiburada.com//vestel-no-frost-kombi-buzdolabi-nfk48001-p

In [68]:
print(len(ALL_PRODUCT_URL))

360


In [70]:
result = []
for details in ALL_PRODUCT_URL[::]:
    html = getAndParseURL(details)
    
    isim = html.find("h1",{"itemprop":"name"}).text.strip()
    price = html.find("span",{"data-bind":"markupText:'currentPriceBeforePoint'"}).text.strip()
    #<span data-bind="markupText:'currentPriceBeforePoint'">7.198</span>()
    satici = html.find("span",class_="seller").a.text.strip()
    #yorumlar = html.find("div",{"id":"comments-container"}).a.span.text
    REVIEW = html.find("a",{"id":"productReviewsTab"}).text.strip().split("(")[1].split(")")[0]
    try:
        Enerji_sinifi = html.find("div",{"id":"productTechSpecContainer"}).find(text="AB Yeni Enerji Sınıfı").findNext().text.strip()
    except:
        Enerji_sinifi = np.nan
    try:
        Aydinlatma = html.find("div",{"id":"productTechSpecContainer"}).find(text="Aydınlatma").findNext().text.strip()
    except:
        Aydinlatma = np.nan
    try:       
        Buz_Matik = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Buz Matik")).findNext().text.strip()
    except:
        Buz_Matik = np.nan    
    try:
        Derinlik_cm = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Derinlik (cm)")).findNext().text.strip()
    except:
        Derinlik_cm = np.nan
    try:
        Dondurucu_Bolme_kapasitesi_Lt = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Dondurucu Bölme Kapasitesi (Lt)")).findNext().text
    except:
        Dondurucu_Bolme_kapasitesi_Lt = np.nan
    try:
        Dondurucu_Bolmeler_Toplam_Net_Hacim = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Dondurucu Bölmeler Toplam Net Hacim")).findNext().text
    except:
        Dondurucu_Bolmeler_Toplam_Net_Hacim = np.nan        
    try:
        Dondurucu_Ozelligi = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Dondurucu Özelliği")).findNext().text
    except:
        Dondurucu_Ozelligi = np.nan     
    try:
        Garanti_Notu = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Garanti Notu")).findNext().text
    except:
        Garanti_Notu = np.nan         
    try:
        Genislik_cm = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Genişlik (cm)")).findNext().text
    except:
        Genislik_cm = np.nan        
    try:
        Hacim_L = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Hacim (L)")).findNext().text
    except:
        Hacim_L = np.nan        
    try:
        Hizli_Dondurma = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Hızlı Dondurma")).findNext().text
    except:
        Hizli_Dondurma = np.nan      
    try:
        Hizli_Sogutma = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Hızlı Soğutma")).findNext().text
    except:
        Hizli_Sogutma = np.nan          
    try:
        Kapi_Acik_Uyarisi = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Kapı Açık Uyarısı")).findNext().text
    except:
        Kapi_Acik_Uyarisi = np.nan          
    try:
        Kontrol_Paneli = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Kontrol Paneli")).findNext().text
    except:
        Kontrol_Paneli = np.nan   
    try:
        Kullanim_Sekli = html.find("div",class_="data-list tech-spec").find(text=re.compile("Kullanım Şekli")).findNext().text
    except: 
        Kullanim_Sekli = np.nan 
    try:
        Renk = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Renk")).findNext().text
    except:
        Renk = np.nan   
    try:
        Ses_Gucu_Duzeyi_Db_A = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Ses Gücü Düzeyi Db(A)")).findNext().text
    except:
        Ses_Gucu_Duzeyi_Db_A = np.nan   
    try:
        Ses_Gucu_Sınıfı = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Ses Gücü Sınıfı")).findNext().text
    except:
        Ses_Gucu_Sınıfı = np.nan  
    try:
        Ses_Seviyesi_dB = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Ses Seviyesi (dB)")).findNext().text
    except:
        Ses_Seviyesi_dB = np.nan  
    try:
        Soğutucu_Bölme_Kapasitesi_Lt = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Soğutucu Bölme Kapasitesi (Lt)")).findNext().text
    except:
        Soğutucu_Bölme_Kapasitesi_Lt = np.nan   
    try:
        Soğutucu_Bölmeler_Toplam_Net_Hacim = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Soğutucu Bölmeler Toplam Net Hacim")).findNext().text
    except:
        Soğutucu_Bölmeler_Toplam_Net_Hacim = np.nan       
    try:
        Su_Buz_Pınarı = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Su & Buz Pınarı")).findNext().text
    except:
        Su_Buz_Pınarı = np.nan    
    try:
        Toplam_Net_Hacim = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Toplam Net Hacim")).findNext().text
    except:
        Toplam_Net_Hacim = np.nan    
    try:
        Uretici = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Üretici")).findNext().text
    except:
        Uretici = np.nan    
    try:        
        Urun_Tipi = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Ürün Tipi")).findNext().text
    except:
        Urun_Tipi = np.nan       
    try:
        Yillik_Enerji_Tuketimi_kWh = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Yıllık Enerji Tüketimi (kWh)")).findNext().text
    except:
        Yillik_Enerji_Tuketimi_kWh = np.nan     
    try:
        Yukseklik_cm = html.find("div",{"id":"productTechSpecContainer"}).find(text=re.compile("Yükseklik (cm)")).findNext().text
    except:
        Yukseklik_cm = np.nan   
    
    
    #Ses_seviyesi = html.find("div",class_="key-properties").find(text=" Ses Seviyesi (dB) ").findNext().text.strip()
    #Hacim = html.find("div",class_="key-properties").find(text=" Hacim (L) ").findNext().text.strip()
    #Garanti =  html.find("tbody",{"th":"AB Yeni Enerji Sınıfı"}).find(text=" AB Yeni Enerji Sınıfı ").findNext().text.strip()

    
    

    
    result.append([isim, price, satici,REVIEW, Enerji_sinifi, Aydinlatma,
Buz_Matik, Derinlik_cm,Dondurucu_Bolme_kapasitesi_Lt,Dondurucu_Bolmeler_Toplam_Net_Hacim,
Dondurucu_Ozelligi, Garanti_Notu,Genislik_cm, Hacim_L,Hizli_Dondurma,Hizli_Sogutma,Kapi_Acik_Uyarisi,Kontrol_Paneli,
Kullanim_Sekli,Renk, Ses_Gucu_Duzeyi_Db_A,Ses_Gucu_Sınıfı,Ses_Seviyesi_dB, Soğutucu_Bölme_Kapasitesi_Lt,
Soğutucu_Bölmeler_Toplam_Net_Hacim, Su_Buz_Pınarı,Toplam_Net_Hacim,Uretici , Urun_Tipi,Yillik_Enerji_Tuketimi_kWh,Yukseklik_cm])
    
    
    kolon_name = ["isim", "price", "satici","REVIEW","Enerji_sinifi","Aydinlatma",
"Buz_Matik", "Derinlik_cm","Dondurucu_Bolme_kapasitesi_Lt","Dondurucu_Bolmeler_Toplam_Net_Hacim", "Dondurucu_Ozelligi", 
"Garanti_Notu","Genislik_cm", "Hacim_L","Hizli_Dondurma","Hizli_Sogutma","Kapi_Acik_Uyarisi","Kontrol_Paneli","Kullanim_Sekli",
"Renk", "Ses_Gucu_Duzeyi_Db_A","Ses_Gucu_Sınıfı","Ses_Seviyesi_dB", "Soğutucu_Bölme_Kapasitesi_Lt",
"Soğutucu_Bölmeler_Toplam_Net_Hacim", "Su_Buz_Pınarı","Toplam_Net_Hacim","Uretici" , "Urun_Tipi","Yillik_Enerji_Tuketimi_kWh",
"Yukseklik_cm"]
df = pd.DataFrame.from_records(result,columns=kolon_name)


In [71]:
df["isim"].value_counts()

Vestel NFK60012 E Gı Pro Wıfı No-Frost Buzdolabı                      3
Samsung RL4353FBASL/TR 473 lt No-Frost Buzdolabı                      3
Vestel NFK52102 Es Wıfı No-Frost Buzdolabı                            3
Siemens KG56NVWF0N 508 lt Alttan Donduruculu No-Frost Buzdolabı       2
Profilo BD2076WFXN 542 L No-Frost Çift Kapılı Buzdolabı               2
                                                                     ..
Profilo BD3056WFUN No-Frost Alttan Donduruculu Buzdolabı              1
Vestel NF52101 S No-Frost Buzdolabı                                   1
Profilo BD2186WFAN 641 L No-Frost Çift Kapılı Buzdolabı               1
Samsung RB44TS134WW/TR 490 lt No-Frost Buzdolabı                      1
Teka CI3 350 NF EU Ankastre Kombi Buzdolabı 275 lt 54 cm 113560005    1
Name: isim, Length: 299, dtype: int64

In [72]:
df

,isim,price,satici,REVIEW,Enerji_sinifi,Aydinlatma,Buz_Matik,Derinlik_cm,Dondurucu_Bolme_kapasitesi_Lt,Dondurucu_Bolmeler_Toplam_Net_Hacim,...,Ses_Gucu_Sınıfı,Ses_Seviyesi_dB,Soğutucu_Bölme_Kapasitesi_Lt,Soğutucu_Bölmeler_Toplam_Net_Hacim,Su_Buz_Pınarı,Toplam_Net_Hacim,Uretici,Urun_Tipi,Yillik_Enerji_Tuketimi_kWh,Yukseklik_cm
0,Samsung RT46K6000WW/TR No-Frost Buzdolabı,6.998,Hepsiburada,848,F,Var,Yok,NaN,NaN,\n113 lt\n,...,\nC\n,NaN,NaN,\n343 lt\n,\nYok\n,\n113 lt\n,\nSamsung\n,\nÇift Kapılı\n,NaN,NaN
1,Vestel No-Frost Kombi Buzdolabı NFK48001 X,7.815,ALISVERİSAVM,133,F,Var,NaN,NaN,NaN,\n115 lt\n,...,\nC\n,NaN,NaN,\n313 lt\n,NaN,\n115 lt\n,\nVestel\n,\nKombi Tipi\n,NaN,NaN
2,Vestel NF45001 No-Frost Buzdolabı,6.175,Zirve - Vestel Yetkili Online Bayi,180,F,Var,NaN,NaN,NaN,\n98 lt\n,...,\nC\n,NaN,NaN,\n305 lt\n,NaN,\n98 lt\n,\nVestel\n,\nÇift Kapılı\n,NaN,NaN
3,Profilo BD2155WFNN 453 L No-Frost Çift Kapılı ...,6.170,BİRTEK AVM,80,F,Var,NaN,NaN,NaN,\n118 lt\n,...,\nC\n,NaN,NaN,\n335 lt\n,NaN,\n118 lt\n,\nProfilo\n,\nÇift Kapılı\n,NaN,NaN
4,Bosch KGN86AIF0N 682 lt No-Frost Buzdolabı,16.401,indirim59,13,E,Var,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,\nYok\n,NaN,\nBosch\n,\nKombi Tipi\n,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Siemens KG56NAIE0N IQ500 508 Lt Buzdolabı 193 ...,19.060,kozmetikfan,0,E,Var,NaN,NaN,NaN,\n108 lt\n,...,\nD\n,NaN,NaN,\n400 lt\n,NaN,\n108 lt\n,\nSiemens\n,\nTek Kapılı\n,NaN,NaN
356,Samsung RF85A911ASL Triple Cooling Teknolojili...,31.771,doludepo,3,E,Var,NaN,NaN,NaN,\n355 lt\n,...,\nC\n,NaN,NaN,\n525 lt\n,NaN,\n355 lt\n,\nSamsung\n,\nGardrop Tipi\n,NaN,NaN
357,Lg GC-B569BLCZ 499LT 70 cm Inox Buzdolabı,21.226,HarikaFiyat,52,E,Var,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,\nYok\n,NaN,\nLG\n,\nKombi Tipi\n,NaN,NaN
358,Franke Fcb 320 Nf Ne F Sınıfı Ankastre Buzdola...,23.254,tamankastrependik,0,F,Var,NaN,NaN,NaN,\n63 lt\n,...,\nC\n,NaN,NaN,\n201 lt\n,NaN,\n63 lt\n,\nFranke\n,\nÇift Kapılı\n,NaN,NaN
